In [1]:
import io
from datetime import datetime, timedelta
from pydantic import BaseModel
from time import sleep

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import berserk
import chess, chess.pgn

In [4]:
import dash

ModuleNotFoundError: No module named 'dash'

## Functions

In [5]:
# Read PGN string into python-chess object
def read_pgn(game_str):
    pgn = io.StringIO(game_str)
    game = chess.pgn.read_game(pgn)
    return game

In [6]:
# Read python-chess object, play and display it in them on a board.
def play_game(game):
    board = game.board()
    for move in game.mainline_moves():
        clear_output(wait=True)
        print(board)
        board.push(move)
        sleep(1)

In [7]:
# Function that returns list of position evaluations for entire game.
# Scores are in centipawn and from White perspective
# If value is None it's because engine saw checkmate in #n
def read_evals(game):
    
    eval_lst = []
    for node in game.mainline():
        if node.eval() != None:
            eval_lst.append(node.eval().white().score())

    return eval_lst

def read_moves(game):
    
    move_lst = [str(node.san()) for node in game.mainline()]
    return(move_lst)

def read_game_eval_recursive(game_node):
    
    if game_node.is_end():
        print(game_node.eval())
        return [None]
    else:
        if game_node.eval()!=None:
            return [game_node.eval().white().score()] + read_game_eval(game_node[0])
        else:
            return read_game_eval(game_node[0])   

In [8]:
class lichess_communication:
    
    lichess_id = None
    API_TOKEN = None
    client = None

    def __init__(self, user) -> None:
        
        # Initialize token
        with open('./token.txt') as f:
            self.API_TOKEN = f.readline()[:-1]
        
        # Initialize lichess client
        self.lichess_id = user
        session = berserk.TokenSession(self.API_TOKEN)
        self.client = berserk.Client(session=session)
    
    def get_games(self, )

SyntaxError: invalid syntax (<ipython-input-8-df4ff0c17e71>, line 18)

## API session setup using berserk

In [15]:
with open('conf/token.txt') as f:
    API_TOKEN = f.readline()[:-1]

In [16]:
session = berserk.TokenSession(API_TOKEN)
client = berserk.Client(session=session)
user_lichess = 'miguel0f'

## API game request settings

In [17]:
last_Xdays = datetime.now()-timedelta(days=100)

In [18]:
user = 'miguel0f'
since = int(berserk.utils.to_millis(last_Xdays))
until = int(berserk.utils.to_millis(datetime.now()))
color = 'black'
as_pgn = True
analysed = True
evals = True
opening = True

In [23]:
games_gen = client.games.export_by_player('miguel0f',
                                    since=since,
                                    analysed=analysed,
                                    evals=evals,
                                    opening=opening)

games_lst = [game for game in games_gen]

In [25]:
games_lst[0]

{'id': 'SsX6oWLU',
 'rated': True,
 'variant': 'standard',
 'speed': 'classical',
 'perf': 'classical',
 'createdAt': datetime.datetime(2022, 7, 9, 21, 35, 50, 943000, tzinfo=datetime.timezone.utc),
 'lastMoveAt': datetime.datetime(2022, 7, 9, 21, 56, 49, 914000, tzinfo=datetime.timezone.utc),
 'status': 'resign',
 'players': {'white': {'user': {'name': 'THE-GOAT-OF-CHESS',
    'id': 'the-goat-of-chess'},
   'rating': 1756,
   'ratingDiff': 16,
   'analysis': {'inaccuracy': 2, 'mistake': 0, 'blunder': 1, 'acpl': 65}},
  'black': {'user': {'name': 'miguel0f', 'id': 'miguel0f'},
   'rating': 1705,
   'ratingDiff': -10,
   'analysis': {'inaccuracy': 3, 'mistake': 0, 'blunder': 2, 'acpl': 101}}},
 'winner': 'white',
 'opening': {'eco': 'C21', 'name': 'Danish Gambit', 'ply': 5},
 'moves': 'e4 e5 d4 exd4 c3 dxc3 Bc4 Qe7 Nxc3 Nf6 Bg5 h6 Bh4 g5 Bg3 Nxe4 Nd5 Qc5 Qe2 Qa5+ b4 Bxb4+ Kf1 Kd8 Qxe4 Re8 Bxc7+ Qxc7 Qxe8+ Kxe8 Nxc7+ Kd8 Nxa8 Nc6 Bxf7 b6 Rb1 Ba6+ Ne2 Ba5 f3 Nd4 Rb2 Bxe2+ Rxe2 Nxe2 Kxe2 K

In [20]:
games_df = {'lichess_id': [],
            'opening': [],
            'color':    [],
            'result':  [],
            'time':    [],
            'moves':   [],
            'evals':    [],
           'oponent': []}

for game in games_lst:   
    
    game_pgn = read_pgn(game)
    
    if game_pgn==None:
        break
    
    games_df['lichess_id'].append(game_pgn.headers['Site'][20:])
    
    games_df['opening'].append(game_pgn.headers['Opening'])
    games_df['color'].append('white' if game_pgn.headers['White']==user_lichess else 'black')
    
    if  game_pgn.headers['Result']=='1/2-1/2':
        games_df['result'].append('draw')
    elif game_pgn.headers['Result']=='1-0' and game_pgn.headers['White']==user_lichess:
        games_df['result'].append('win')
    elif game_pgn.headers['Result']=='0-1' and game_pgn.headers['Black']==user_lichess:
        games_df['result'].append('win')
    else:
        games_df['result'].append('loss')
    
    games_df['time'].append(game_pgn.headers['TimeControl'])
    
    games_df['moves'].append(read_moves(game_pgn))
    games_df['evals'].append(read_evals(game_pgn))
    
    if game_pgn.headers['White']==user_lichess:
        games_df['oponent'].append(game_pgn.headers['Black'])
    else:
        games_df['oponent'].append(game_pgn.headers['White'])
    
games_df = pd.DataFrame.from_dict(games_df, orient='columns')

In [27]:
games_df['result'].value_counts()

loss    92
win     83
draw     4
Name: result, dtype: int64

In [223]:
games_df.loc[games_df['result']=='loss']

,lichess_id,opening,color,result,time,moves,evals,oponent
1,5upFAHQK,Englund Gambit Complex: Hartlaub-Charlick Gambit,black,loss,900+5,"[d4, e5, dxe5, d6, exd6, Bxd6, e3, Nc6, Nc3, Q...","[0, 174, 144, 146, 117, 162, 95, 89, 95, 100, ...",igood_kz
2,cc981H5L,Elephant Gambit,black,loss,900+30,"[e4, e5, Nf3, d5, Nxe5, dxe4, Qe2, Nf6, Nc3, B...","[33, 12, 19, 123, 76, 82, 0, 28, -13, 25, -64,...",RoyStorey
3,TYfBJ0VR,Scandinavian Defense: Main Line,white,loss,900+30,"[e4, d5, exd5, Qxd5, Nc3, Qa5, Nf3, Nf6, d4, c...","[33, 82, 66, 75, 71, 63, 57, 75, 80, 70, 49, 1...",wwweniy
4,J18cNKZ7,Elephant Gambit: Paulsen Countergambit,black,loss,600+5,"[e4, e5, Nf3, d5, exd5, e4, Qe2, Nf6, Nc3, Be7...","[33, 12, 19, 123, 130, 140, 136, 119, 95, 138,...",Shura73
6,JGLFwxP1,Scotch Game,white,loss,600+5,"[e4, e5, Nf3, Nc6, d4, d6, Bc4, h6, O-O, Bg4, ...","[33, 12, 19, 32, 13, 97, 28, 99, 85, 134, -125...",joker007stcherek
...,...,...,...,...,...,...,...,...
178,fQMO9TnH,Elephant Gambit: Paulsen Countergambit,black,loss,1800+0,"[e4, e5, Nf3, d5, exd5, e4, Ne5, Qxd5, Nc4, Nf...","[33, 12, 19, 123, 130, 140, 20, 23, -2, 0, -82...",driss66
179,R0szawvJ,Sicilian Defense: Smith-Morra Gambit,white,loss,2700+30,"[e4, c5, d4, b6, Nf3, Nf6, Nc3, Bb7, Qd3, Nc6,...","[33, 32, 0, 167, 82, 216, 96, 128, 109, 275, 0...",bacco1966
180,s7dNRgAM,Elephant Gambit: Paulsen Countergambit,black,loss,7200+20,"[e4, e5, Nf3, d5, exd5, e4, Ne5, Qxd5, Ng4, Bc...","[33, 12, 19, 123, 130, 140, 20, 52, -65, 5, -3...",klyushonok
181,fhojTjMQ,Elephant Gambit,black,loss,7200+20,"[e4, e5, Nf3, d5, Nxe5, dxe4, d3, Bd6, Bf4, Qe...","[33, 12, 19, 123, 76, 82, 13, 52, -319, -41, -...",Ahmdjokar_1402


In [26]:
game = read_pgn(games_lst[12])
game_eval = games_df.iloc[12]
play_game(game)

plt.figure()
plt.plot(game_eval)
plt.show()

TypeError: initial_value must be str or None, not dict

## Code snippets to look into data returned by API

In [ ]:
for k in range(len(a)):
    if a[k]['players'][color]['user']['name']!='miguel0f':
        print('ahahah')